# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 37](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=37)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "true"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=False,                # TODO change to False before push
    limit=limit,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-04 to 2024-11-05 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac)
* [Summary plots of whole night](#Summary-plots-of-whole-night)
* [Time Accounting](#Time-Accounting)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log)
* [Narrative Log](#Narrative-Log)
* [Developer Only Section](#Developer-Only-Section)

## Night log (summary) <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: Simonyi

```
We started with hexapod warmups while Felipe and Christian worked inside the M1M3 cell for leaks. We still have 8 actuator leaking but it is not critical to stop the M1M3 test. There was M2hexapod warmup script update from Te-Wei's request. 

We ran ComCam USDF Checkout to take multiple bias and dark images. Lights were on.

Dome handover and TMA handover both happened at 3pm. 
During dome handover, we had to reset cRIO once after switching from local mode to remote mode, according to Wouter's comment. Dome was moving fine but we had trouble with telemetry not read. We suspected the DDS issue, but Tiago confirmed it wasn't, and was solved by Wouter's update and recycling the MTDome.

We ran bump test with secondary FA 330 failed, and then HP breakaway test.

HVAC setpoint was changed multiple times during the day, according to the temperature difference between the ambient and M2 air temperature (inside dome, outside M1M3 cell temp). MTM1M3TS had hearbeat lost twice, but 2nd time was due to changing the fan speed without entering engineering mode. We recovered (reset) and set to 300RPM (put 30 as constant).

Outside temperature and M2 air temperature was within 2C difference, we decided to turn off HVAC and open the dome. There was miscommunication between trying to do other setups and TMA checkout first and opening the dome, and since we had TMA problem with positive limit switch blocking the elevation drive from turning on, due to wrong position readout (>90 deg), we decided to open the dome before TMA checkout is ready. 

TMA positive limit switch problem solved by ignoring the limit switch and turn on the elevation drive (reading out correct position) then move to el=80. After that we switched to CSC mode, but failed again with wrong elevation readout. We recovered, and were able to enable the MTMount, when correct elevation was readout. TMA checkout was done with several small issues such as typo in the configuration, mis-instruction on MTDome, MTDomeTrajectory, etc.

We moved TMA with move_p2p to az=160, el=65 to have same position as last week to compare the alignment at the same position.  


------- AOS Night Summary -------
We opened the dome shutter around 9:00 pm and started T88 alignment with the laser tracker by 10:30-10:45 pm, despite a few dome faults. After alignment, we took a sky image to confirm positioning, and initial focus looked promising. A follow-up triplet image provided a closer focus check, though Tiago had to update MTCS scripts due to some failures.

While the laser alignment helped, some spherical aberration remained. We refined focus by sweeping in +Cam_dz and +M2_dz, reaching around 1.7 arcsec. At this point we took a series of images to check the variance of the PSF measurement, it seems it oscillates +/- 0.6 arcsec. Then, by adjusting +Cam_dz and -M2_dz, we minimized the spherical aberration and recorded this optimal focus position for tomorrow, although the PSF was worse. Our new script set_dof.py now lets us revert to this focus point, resolving previous week's confusion with compensated versus uncompensated positions—a very handy improvement tonight. 

We also tried to minimize coma with an M2 Ry sweep, but CCS issues made it inconclusive. Moving to closed-loop testing with hexapods, we found wavefront estimation pipeline speed is becoming a bottleneck; RA should hopefully help improve this soon. We ran four closed-loop iterations with hexapods, adding astigmatism on the last step, but it appears the loop is doubling in the wrong direction, indicating a possible sign error to investigate tomorrow.

The final closed-loop correction produced unrealistic results due to an outlier sensor Zernike estimate at the central sensor, likely from the laser. We'll need to investigate this further and consider a safeguard in the wavefront estimation pipeline. Overall, it was a successful and educational night. By the way, someone promised to sing when we reach 1.4 arcsec… guess who!

===== After Closing Dome Summary == 

We started closing dome around 03:15am with closing M1M3 mirror covers and dome shutter. We had an issue while we closed the dome shutter - the left aperture was not closed totally and kept closing even after the right aperture was closed. We stopped closing and opened the aperture a little bit and closed it again. It takes more than a few minutes and then it seemed closed (no light on the buttons blinked) but night crew heard unusual big noise from cabinet. Need further checks if there is no issue on dome slit and it is closed without issue. HVAC turning back to daytime move and set point kept in 9C. Then we run BLOCK-T220, ComCam Daytime Calibrations until 9am UTC/06 am UTC. We parked the TMA on horizon with raised M1M3 per Petr K.'s request. 

Two OBS ticket filed - OBS-646 (MTDome AMCS fault) and OBS-647 (issue with dome shutter closing). 


* Please note that there was Keith Bechtol as a Late shit observer.
```

Telescope Status: TMA parked at El: 0.0deg (Horizon), Az: -32 deg as requested. Locking pins were engaged, OSS on, PS OFF, and the mirror cover retracted (opened). Dome parked at 328 deg. All CSCs are set as requested:
MTM1M3 DISABLED (lowered), MTM2 enabled in closed loop, MTAOS STANDBY, MTDome DISABLED, MTDomeTrajectory Standby, MTMount Disabled, MTHexapod1,2, Rotator in Standby with PXIs and Cabinets turned off. MTPtg enabled. 

*Authors: HyeYun Park, Kshitija Kelkar, Yijung Kang, Sandrine Thomas, Chris Walter*

```
We started with hexapod warmups while Felipe and Christian worked inside the M1M3 cell for leaks. We still have 8 actuator leaking but it is not critical to stop the M1M3 test. There was an M2hexapod warmup script update from Te-Wei's request. 

We ran ComCam USDF Checkout to take multiple bias and dark images. The lights were on.

Dome handover and TMA handover both happened at 3pm. During the dome handover, we had to reset cRIO once after switching from local mode to remote mode, according to Wouter's comment. Dome was moving fine but we had trouble with telemetry not read. We suspected the DDS issue, but Tiago confirmed it wasn't, and was solved by recycling the MTDome.

We ran a bump test with secondary FA 330 failed, and then the HP breakaway test.

The HVAC setpoint was changed multiple times during the day, according to the temperature difference between the ambient and M2 air temperature (inside dome, outside M1M3 cell temp). MTM1M3TS had heartbeat lost twice, but 2nd time was due to changing the fan speed without entering engineering mode. We recovered (reset) and set it to 300RPM (put 30 as constant).

Outside temperature and M2 air temperature were within 2C difference, we decided to turn off HVAC and open the dome. There was a miscommunication between trying to do other setups and TMA checkout first and opening the dome, and since we had TMA problem with the positive limit switch blocking the elevation drive from turning on, due to wrong position readout (>90 deg), we decided to open the dome before TMA checkout is ready. 

The TMA positive limit switch problem was solved by ignoring the limit switch and turn on the elevation drive (reading out the correct position) then moving to el=80. After that, we switched to CSC mode but failed again with the wrong elevation readout. We recovered and were able to enable the MTMount when the correct elevation was readout. TMA checkout was done with several small issues such as a typo in the configuration, mis-instruction on MTDome, MTDomeTrajectory, etc.

We moved TMA with move_p2p to az=160, el=65 to have the same position as last week to compare the alignment at the same position.
```

Telescope Status: 

*Authors: HyeYun Park, Kshitija Kelkar, Yijung Kang, Sandrine Thomas, Chris Walter*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-04**

,(times in UTC),
Moon Rise,2024-11-04 11:35:23.572,
Moon Set,2024-11-04 01:22:18.808,
Moon Illumination,7%,(% illuminated)
Astronomical Twilight (morning),2024-11-04 08:17:33.018,(-18 degrees)
Astronomical Twilight (evening),2024-11-04 00:35:13.375,(-18 degrees)
Solar Midnight,2024-11-04 04:26:23.324,
Nautical Twilight (morning),2024-11-04 08:49:10.466,(-12 degrees)
Nautical Twilight (evening),2024-11-04 00:03:35.802,(-12 degrees)
Civil Twilight (morning),2024-11-04 09:19:34.997,(-6 degrees)
Civil Twilight (evening),2024-11-03 23:33:11.094,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.

In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCamSim,LATISS,LSSTComCam,LSSTCam
Total Night,7:42:19,7:42:19,7:42:19,7:42:19
Total Exposure,0:00:00,0:00:00,1:02:38,5:05:32
Slew time(1),0:00:00,0:00:00,0:00:00,0:00:00
Readout time(2),0:00:00,0:00:00,0:06:40,0:47:11
Time loss to fault,NA,NA,NA,NA
Time loss to weather,NA,NA,NA,NA
Idle time,7:42:19,7:42:19,6:39:40,2:36:46
Number of exposures,0,0,174,1231
Mean readout time,0.000639,0.000639,0.000639,0.000639
Number of slews(1),0,0,0,0


- (1) Expected slew per Schedular
- (2) There is no practical way to get detect read time.  Value of 2.3 seconds inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241104&max_day_obs=20241105) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
(TODO: BLOCKS observed) 

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

NOTE: *modified all_sources.py:uniform_field_counts()*
NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

Tally exposures with specific values of selected fields

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 174 total exposures**

observation_reason

...

,flat,bias
0,1221,10


science_program

,E2016,unknown
0,1221,10


observation_type

,flat,bias,dark
0,638,329,264


### (change) <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
#### TODO According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument. (that tally has been added below)
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

### Instrument: LSSTComCam (9)

* CC_O_20241104_000021 : 2024-11-05T03:08:31.113045
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000022 : 2024-11-05T03:08:31.152648
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000023 : 2024-11-05T03:08:31.189583
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000024 : 2024-11-05T03:08:31.228502
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000025 : 2024-11-05T03:08:31.266365
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000026 : 2024-11-05T03:08:31.069815
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`These images are not useful for the particular block as we had vignetted donut and then the pointing component failed.`

* CC_O_20241104_000127 : 2024-11-05T06:20:43.256294
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`The Block should have been Block-T60 instead of Block-T249`

* CC_O_20241104_000128 : 2024-11-05T06:20:43.218049
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`The Block should have been Block-T60 instead of Block-T249`

* CC_O_20241104_000129 : 2024-11-05T06:20:43.177427
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`The Block should have been Block-T60 instead of Block-T249`

Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

**LSSTComCam 9 Exposures with messages**

good,questionable,junk,none
0,0,0,9


## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [16]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-04 04:04:34.320647** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Block T92 with vector [+Cam dz, +M2 dz], we use a parameter march to do the seep in this vector. We are not stepping through only one direction but two at a time, because of their combined effect</pre>


- **2024-11-04 03:57:51.312522** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>No acknowledgement on the camera cable wrap following on first try. This was the same error seen last time when there were DDS issues. We tried again and it passed.</pre>


- **2024-11-04 03:56:57.828648** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We are now starting manual alignment with laser on. We'll do a parameter march in Cam dz and M2 dz.</pre>


- **2024-11-04 03:31:51.385031** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Right now we got the first measurement from laser on Camera for BLOCK-T88. All the previous laser measurements correspond to BLOCK-T222</pre>


- **2024-11-04 03:18:15.157159** Time Lost: 3.0; Time Lost Type: weather;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>David is going to open the rest of the way and we will begin on sky testing. End of weather loss.</pre>


...

- **2024-11-04 16:59:52.799013** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>position in x reached a max 6500 um as well for M2Hex</pre>


- **2024-11-04 16:41:18.568916** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>So the Block description says that each of the axes x-y will reach a max of 4500 um for M2Hex but I notice that the position in y reached max 6500 um.</pre>


- **2024-11-04 16:34:07.856570** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2Hex exercising in y-axis !</pre>


- **2024-11-04 16:24:02.092631** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2Hex ENABLED and in between warm-ups</pre>


- **2024-11-04 16:21:50.310134** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>CamHexapod ENABLED and in between warm-ups 
M2Hex in DISABLED-- but Bruno has updated the warm-up script so that will be done next</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px"> REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [17]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-04 12:00:00**.
- min_dayobs='2024-11-04', max_dayobs='2024-11-05'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [18]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,2,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241104&max_day_obs=20241105
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241104&max_day_obs=20241105
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241104&max_day_obs=20241105
exposurelog/exposures.LSSTComCam,174,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241104&max_day_obs=20241105
exposurelog/exposures.LSSTCam,1231,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241104&max_day_obs=20241105
exposurelog/messages,9,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241104&max_day_obs=20241105
narrativelog/messages,52,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-04T00%3A00%3A00&max_date_added=2024-11-05T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [19]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [20]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241104&max_day_obs=20241105

None
- Got 2 records.  


### Overview for Service: `exposurelog` [9]

...

None
- Got 9 records.  


### Overview for Service: `narrativelog` [52]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-04T00%3A00%3A00&max_date_added=2024-11-05T00%3A00%3A00

None
- Got 52 records.  


In [21]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-05 18:12:51.252863
